In [1]:
import torch
print(torch.__version__)      # 2.3.0+cu121
print(torch.version.cuda)     # 12.1
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA GeForce RTX 4090


2.3.0+cu121
12.1
True
NVIDIA GeForce RTX 4090


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

c:\Users\ayato\anaconda3\envs\llama7b\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
print("▶ モデルをダウンロード＆ロード中…（初回のみ数分）")
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"  
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token      # Warning 回避

print(f"✅ ロード完了・使用 VRAM ≈ {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

▶ モデルをダウンロード＆ロード中…（初回のみ数分）


c:\Users\ayato\anaconda3\envs\llama7b\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ayato\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.59s/it]


✅ ロード完了・使用 VRAM ≈ 7.24 GB


In [10]:
messages = [
    {"role": "system",    "content": "you are AI assistant"},
    {"role": "user",      "content": "Could you make a new idea about how to tune LLM by learning bisociation ?"},
]
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

out = model.generate(
    inputs,
    max_new_tokens=12000,
    temperature=0.4,
    top_p=0.8,
    repetition_penalty=1.15,
    eos_token_id=tokenizer.eos_token_id,
)
print(tokenizer.decode(out[0], skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] you are AI assistant

Could you make a new idea about how to tune LLM by learning bisociation ? [/INST] Bisociation is the ability to combine and integrate multiple frames of reference or meaning into a single mental construct. In the context of Language Model (LLM) tuning, we can explore the following idea:

1. Multimodal Input: Instead of relying solely on textual data for training our LLM, we can incorporate multimodal inputs such as images, audio, and video. This will allow the model to learn associations between words and their visual, auditory, or other sensory representations. For example, given an input like "a red apple," the model should be able to generate not only the correct word sequence but also produce a visually accurate representation of an apple.

2. Concept Net Integration: We can build a concept net that captures relationships between different concepts and use it to guide the learning process in our LLM. The concept net would represent various semantic rela